In [1]:
from PIL import Image, ImageEnhance, ImageFilter, ImageDraw
import random
import numpy as np
import cv2
import os
from skimage.util import random_noise

In [7]:
def generate_realistic_variations(image_path, output_path):
    img = Image.open(image_path)
    
    # Ensure the image has a white background (convert transparent/black to white)
    img = img.convert('RGBA')
    white_bg = Image.new('RGBA', img.size, (255, 255, 255, 255))
    img = Image.alpha_composite(white_bg, img)
    img = img.convert('L')  # Convert to grayscale
    
    # Base modifications that should always be applied
    modifications = [
        'brightness', 'contrast', 'rotate', 'resize', 
        'noise', 'thickness', 'partial_occlusion'
    ]
    
    # Apply 4-6 random modifications (more variability than before)
    selected_mods = random.sample(modifications, random.randint(4, 6))
    
    for mod in selected_mods:
        try:
            if mod == 'brightness':
                enhancer = ImageEnhance.Brightness(img)
                img = enhancer.enhance(random.uniform(0.7, 1.5))  # Wider range
            
            elif mod == 'contrast':
                enhancer = ImageEnhance.Contrast(img)
                img = enhancer.enhance(random.uniform(0.7, 1.5))
            
            elif mod == 'rotate':
                angle = random.uniform(-15, 15)  # Slightly larger rotation
                # Rotate on white background
                white_bg = Image.new('L', img.size, 255)
                img = img.rotate(angle, expand=True, resample=Image.BICUBIC, fillcolor=255)
                # If rotation expanded the image, composite on white
                if angle != 0:
                    new_size = img.size
                    new_bg = Image.new('L', new_size, 255)
                    position = ((new_size[0]-img.size[0])//2, (new_size[1]-img.size[1])//2)
                    new_bg.paste(img, position)
                    img = new_bg
            
            elif mod == 'resize':
                scale_factor = random.uniform(0.8, 1.5)  # More constrained range
                width, height = img.size
                new_width = int(width * scale_factor)
                new_height = int(height * scale_factor)
                img = img.resize((new_width, new_height), Image.LANCZOS)
                # After resize, put on white canvas of original size
                if scale_factor != 1.0:
                    original_size = Image.open(image_path).size
                    white_bg = Image.new('L', original_size, 255)
                    position = ((original_size[0]-new_width)//2, (original_size[1]-new_height)//2)
                    white_bg.paste(img, position)
                    img = white_bg
            
            elif mod == 'noise':
                # Convert to array for noise addition
                img_array = np.array(img)
                if len(img_array.shape) == 2:  # Grayscale
                    img_array = random_noise(img_array, mode='gaussian', var=0.001)
                img = Image.fromarray((img_array * 255).astype(np.uint8))
                # Ensure background stays white
                img = img.point(lambda p: 255 if p > 200 else p)
            
            elif mod == 'thickness':
                # Simulate variable stroke thickness
                img_array = np.array(img)
                if len(img_array.shape) == 2:  # Grayscale
                    kernel_size = random.randint(1, 3)
                    img_array = cv2.dilate(img_array, 
                                         np.ones((kernel_size,kernel_size), np.uint8),
                                         iterations=1)
                    img = Image.fromarray(img_array)
                # Ensure background stays white
                img = img.point(lambda p: 255 if p > 200 else p)
            
            elif mod == 'partial_occlusion':
                # Add random occlusion patches - only white rectangles now
                draw = ImageDraw.Draw(img)
                width, height = img.size
                for _ in range(random.randint(1, 3)):
                    x1 = random.randint(0, width//4)
                    y1 = random.randint(0, height//4)
                    x2 = random.randint(x1, width//2)
                    y2 = random.randint(y1, height//2)
                    draw.rectangle([x1,y1,x2,y2], fill=255)  # Only white occlusion
        
        except Exception as e:
            print(f"Error applying {mod}: {e}")
            continue
    
    # Final processing to ensure white background
    img = img.convert('L')
    # Binarize while preserving white background
    img = img.point(lambda p: 255 if p > 200 else 0)
    
    # Save with quality variation
    quality = random.randint(70, 95)
    img.save(output_path, quality=quality)

In [23]:
from os import listdir
from os.path import isfile, join
figures = [f for f in listdir('etalon') if isfile(join('etalon', f))]
figures

['air1.png',
 'air2.png',
 'air3.png',
 'earth1.png',
 'earth2.png',
 'earth3.png',
 'fire1.png',
 'fire2.png',
 'fire3.png',
 'water1.png',
 'water2.png',
 'water3.png']

In [29]:
from  collections import Counter

cnt = Counter()
mx = -1
files = dict()
for filename in listdir('images'):
        if filename.endswith('.png'):
            
            if filename.startswith('x'):
                continue
            else:
                label = filename[:filename.find('-')]
                if label not in files.keys():
                    files[label] = []
                files[label].append(filename[filename.find('-'):])
                cnt[label] += 1
                if cnt[label] > mx:
                    mx = cnt[label]
print(cnt)


Counter({'air1': 402, 'air2': 402, 'air3': 402, 'earth1': 402, 'earth2': 402, 'earth3': 402, 'fire1': 402, 'fire2': 402, 'fire3': 402, 'water1': 402, 'water2': 402, 'water3': 402})


In [28]:

for element in cnt:
    while cnt[element] <= mx:
        cnt[element] += 1
        output_name = element + '-' + str(cnt[element] + 3000) + '.png'
        #print(os.path.join('images', element + files[element][random.randint(0, len(files[element]) - 1)]),)
        generate_realistic_variations(
           os.path.join('images', element + files[element][random.randint(0, len(files[element]) - 1)]),
           os.path.join('images', output_name)
        )
        

In [8]:

for i, figure in enumerate(figures):
    for j in range(150):  # 150 variations per figure
        output_name = f"{figure.split('.')[0]}-{j}.png"
        generate_realistic_variations(
            os.path.join('etalon', figure),
            os.path.join('images', output_name)
        )